<a href="https://colab.research.google.com/github/ShraddhaSharma24/-Multimodal-Emotion-Recognition-with-Time-Series-NLP/blob/main/multimodal_emotion_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install Kaggle API (if not installed)
!pip install kaggle

# Authenticate with Kaggle (One-Time Step)
from google.colab import files
files.upload()  # Upload kaggle.json (API Key)

# Set up Kaggle API (Only do this once)
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json



Saving kaggle.json to kaggle.json


In [3]:
from google.colab import files

# Upload train and test CSV files
uploaded = files.upload()



Saving train_sent_emo.csv to train_sent_emo.csv
Saving test_sent_emo.csv to test_sent_emo.csv


In [4]:
import pandas as pd
import re
from sklearn.preprocessing import LabelEncoder

# Define dataset paths
train_path = "/content/train_sent_emo.csv"
test_path = "/content/test_sent_emo.csv"

# Load datasets
train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

# Function to clean text (remove special characters, extra spaces)
def clean_text(text):
    text = re.sub(r"[^a-zA-Z0-9.,!?']+", ' ', text)  # Keep punctuation
    return text.strip().lower()

# Apply cleaning
train_data["Utterance"] = train_data["Utterance"].apply(clean_text)
test_data["Utterance"] = test_data["Utterance"].apply(clean_text)

# Encode labels (Convert emotion categories into numerical values)
label_encoder = LabelEncoder()
train_data["Emotion"] = label_encoder.fit_transform(train_data["Emotion"])
test_data["Emotion"] = label_encoder.transform(test_data["Emotion"])

# Display cleaned data


In [5]:
print("Processed Train Data:")
print(train_data.head())

Processed Train Data:
   Sr No.                                          Utterance          Speaker  \
0       1  also i was the point person on my company s tr...         Chandler   
1       2                   you must ve had your hands full.  The Interviewer   
2       3                            that i did. that i did.         Chandler   
3       4      so let s talk a little bit about your duties.  The Interviewer   
4       5                              my duties? all right.         Chandler   

   Emotion Sentiment  Dialogue_ID  Utterance_ID  Season  Episode  \
0        4   neutral            0             0       8       21   
1        4   neutral            0             1       8       21   
2        4   neutral            0             2       8       21   
3        4   neutral            0             3       8       21   
4        6  positive            0             4       8       21   

      StartTime       EndTime  
0  00:16:16,059  00:16:21,731  
1  00:16:21,940  0

In [6]:
import torch
from transformers import BertTokenizer

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize text
train_encodings = tokenizer(list(train_data["Utterance"]), padding=True, truncation=True, return_tensors="pt")
test_encodings = tokenizer(list(test_data["Utterance"]), padding=True, truncation=True, return_tensors="pt")

# Convert labels to tensors
train_labels = torch.tensor(train_data["Emotion"].values)
test_labels = torch.tensor(test_data["Emotion"].values)

print("Tokenized Sample:", train_encodings.input_ids.shape)
print("Labels Sample:", train_labels.shape)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Tokenized Sample: torch.Size([9989, 95])
Labels Sample: torch.Size([9989])


In [7]:
import torch
from torch.utils.data import Dataset, DataLoader

# Define a PyTorch Dataset class
class MELDDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item

# Create DataLoader for training and testing
train_dataset = MELDDataset(train_encodings, train_labels)
test_dataset = MELDDataset(test_encodings, test_labels)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

print("Dataset & Dataloader ready!")


Dataset & Dataloader ready!


In [8]:
from transformers import BertForSequenceClassification

# Define BERT model for sequence classification
num_labels = len(set(train_data["Emotion"]))  # Number of unique emotion classes

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)

print("BERT Model Initialized with", num_labels, "Emotion Classes")


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERT Model Initialized with 7 Emotion Classes


In [9]:
from transformers import AdamW

# Define optimizer & loss function
optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# Move model to GPU (if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print("Model moved to:", device)


/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Model moved to: cuda


In [10]:
from transformers import get_scheduler
import torch.nn as nn

# Learning rate scheduler
num_training_steps = len(train_loader) * 3  # 3 epochs
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# Training loop
epochs = 3
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        batch = {key: val.to(device) for key, val in batch.items()}

        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}")

print("Training Completed!")


Epoch 1, Loss: 1.1858
Epoch 2, Loss: 0.9231
Epoch 3, Loss: 0.6887
Training Completed!


In [11]:
from sklearn.metrics import accuracy_score, classification_report

# Evaluate the model
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for batch in test_loader:
        batch = {key: val.to(device) for key, val in batch.items()}
        outputs = model(**batch)

        preds = torch.argmax(outputs.logits, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(batch["labels"].cpu().numpy())

# Compute accuracy & classification report
accuracy = accuracy_score(all_labels, all_preds)
report = classification_report(all_labels, all_preds, target_names=label_encoder.classes_)

print(f"Test Accuracy: {accuracy:.4f}")
print("Classification Report:\n", report)


Test Accuracy: 0.6406
Classification Report:
               precision    recall  f1-score   support

       anger       0.49      0.44      0.47       345
     disgust       0.38      0.12      0.18        68
        fear       0.45      0.18      0.26        50
         joy       0.59      0.61      0.60       402
     neutral       0.75      0.82      0.79      1256
     sadness       0.38      0.25      0.30       208
    surprise       0.51      0.61      0.56       281

    accuracy                           0.64      2610
   macro avg       0.51      0.43      0.45      2610
weighted avg       0.62      0.64      0.63      2610



In [13]:
!pip install datasets


In [14]:
from datasets import list_datasets

# Search for AMIGOS or similar datasets
available_datasets = list_datasets()
print([d for d in available_datasets if "eeg" in d.lower() or "emotion" in d.lower()])

ImportError: cannot import name 'list_datasets' from 'datasets' (/usr/local/lib/python3.11/dist-packages/datasets/__init__.py)